In [6]:
# params
concat_title = True
window = 5 # how many pages you want to include in one doc
book_included = False
booktitle = 'thebook' # if you have abook this is its name
exampath = './midterm exam/Midterm exam.html' # if you want to check



In [2]:
path = './study_material/'
filenames = open(path+'filenames.txt','r').read().split('\n')
filenames.remove('')
filenames = list(map(lambda x: path +x, filenames)) #"\'" +x+"\'"
filenames

['./study_material/Lecture 10: Information Retrieval and Web Search Text Classification.pdf',
 './study_material/Lecture 12: Clustering and Latent Semantic Indexing.pdf',
 './study_material/Lecture 14: Support vector machines and machine learning on documents.pdf',
 './study_material/Lecture 1: Introduction to IR. Boolean.pdf',
 './study_material/Lecture 2: The term vocabulary and postings lists.pdf',
 './study_material/Lecture 3: Dictionaries and tolerant retrieval.pdf',
 './study_material/Lecture 4: Index construction.pdf',
 './study_material/Lecture 5: Index compression.pdf',
 './study_material/Lecture 6: Ranking, vector space.pdf',
 './study_material/Lecture 7: Evaluation.pdf',
 './study_material/Lecture 9: Relevance feedback and query expansion.pdf',
 './study_material/thebook.pdf']

# Go through the docs and make other docs


## window 

how many pages will be concated together, similar to stride





In [3]:
from PyPDF2 import PdfFileReader


docs = []
for filename in filenames:
    stream = open(filename,'rb')
    x = PdfFileReader(stream=stream)
    num_pages = x.getNumPages()
    print(filename[17:], num_pages)
    
    
    
    for i in range(1,1+num_pages-window):
        
        if filename[17:].find(booktitle) != -1 and not book_included:
            continue
        concateddocs = [filename[17:] + x.getPage(j).extractText() for j in range(i,i+window)]
        docs.append('\n'.join(concateddocs))
docs = list(map(lambda x:x.replace('\n',' '),docs))
docs = list(map(lambda x:x.replace('  ',' '),docs))

print(len(docs))



Lecture 10: Information Retrieval and Web Search Text Classification.pdf 48
Lecture 12: Clustering and Latent Semantic Indexing.pdf 104
Lecture 14: Support vector machines and machine learning on documents.pdf 62
Lecture 1: Introduction to IR. Boolean.pdf 50
Lecture 2: The term vocabulary and postings lists.pdf 54
Lecture 3: Dictionaries and tolerant retrieval.pdf 42
Lecture 4: Index construction.pdf 43
Lecture 5: Index compression.pdf 44
Lecture 6: Ranking, vector space.pdf 44
Lecture 7: Evaluation.pdf 49
Lecture 9: Relevance feedback and query expansion.pdf 49
thebook.pdf 581
534


# try Word2vec


try before the pretrained embeddings

In [4]:
import pandas as pd
import re
import logging
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)


# pre processing data
def cleanData(sentence):
    # convert to lowercase, ignore all special characters - keep only
    # alpha-numericals and spaces
    sentence = re.sub(r'[^A-Za-z0-9\s]', r'', str(sentence).lower())

    # remove stop words
    sentence = " ".join([word for word in sentence.split()
                        if word not in stopwords.words('english')])

    return sentence


In [5]:
df = pd.DataFrame(dict(zip(['id','doc'], [[i for i in range(len(docs))],docs])))
df.to_csv('./data/QanatovCSVed.csv')

In [6]:


df = pd.read_csv('./data/QanatovCSVed.csv', encoding='ISO-8859-1', index_col=0)

# drop duplicate rows
df = df.drop_duplicates(subset='doc')

# clean data
df['doc'] = df['doc'].map(lambda x: cleanData(x))

# get array of titles
docs = df['doc'].values.tolist()

# tokenize the each title
tok_docs = [word_tokenize(doc) for doc in docs]

# refer to here for all parameters:
# https://radimrehurek.com/gensim/models/word2vec.html
model = Word2Vec(tok_docs, sg=1, size=100, window=5, min_count=5, workers=4,
                 iter=100)

# save model to file
model.save('./data/Qanatov_solver.model')



2020-05-10 11:54:58,046 : INFO : collecting all words and their counts
2020-05-10 11:54:58,047 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-10 11:54:58,064 : INFO : collected 4694 word types from a corpus of 90718 raw words and 534 sentences
2020-05-10 11:54:58,065 : INFO : Loading a fresh vocabulary
2020-05-10 11:54:58,162 : INFO : effective_min_count=5 retains 4281 unique words (91% of original 4694, drops 413)
2020-05-10 11:54:58,163 : INFO : effective_min_count=5 leaves 89740 word corpus (98% of original 90718, drops 978)
2020-05-10 11:54:58,191 : INFO : deleting the raw counts dictionary of 4694 items
2020-05-10 11:54:58,192 : INFO : sample=0.001 downsamples 56 most-common words
2020-05-10 11:54:58,194 : INFO : downsampling leaves estimated 76711 word corpus (85.5% of prior 89740)
2020-05-10 11:54:58,213 : INFO : estimated required memory for 4281 words and 100 dimensions: 5565300 bytes
2020-05-10 11:54:58,214 : INFO : resetting layer weights


In [7]:
from bs4 import BeautifulSoup
from QanatovSolver import QanatovSolver


page = open(exampath, 'r').read()
soup = BeautifulSoup(page, 'html.parser')
questions = soup.find_all('div', attrs={'class': 'qtext'})
questions = list(questions)
print(questions[0].text)
answers = soup.find_all('div', attrs={'class': 'ablock'})
answers = list(answers)
qanatov_solver = QanatovSolver()
for answer, question in zip(answers, questions):
    children = list(answer.children)[1].children
    # print(*list(children),sep='\n\n\n')
    for child in list(children):
        try:
            score = qanatov_solver.get_similarity(question.text, child.label.text)
            child.label.string = child.label.text + f' :: Score is {score}' +f" Class:{child.attrs['class']}"
            print('modified')
        except:
            continue

with open("./exam_output.html", "wb") as f_output:
    f_output.write(soup.prettify("utf-8"))

2020-05-10 14:02:00,259 : INFO : loading Word2Vec object from ./data/Qanatov_solver.model
2020-05-10 14:02:00,323 : INFO : loading wv recursively from ./data/Qanatov_solver.model.wv.* with mmap=None
2020-05-10 14:02:00,324 : INFO : setting ignored attribute vectors_norm to None
2020-05-10 14:02:00,325 : INFO : loading vocabulary recursively from ./data/Qanatov_solver.model.vocabulary.* with mmap=None
2020-05-10 14:02:00,326 : INFO : loading trainables recursively from ./data/Qanatov_solver.model.trainables.* with mmap=None
2020-05-10 14:02:00,330 : INFO : setting ignored attribute cum_table to None
2020-05-10 14:02:00,331 : INFO : loaded ./data/Qanatov_solver.model


What is information retrieval?
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modified
modifi